# Welcome to Google Colab!
This is the environment that we'll be using to simulate our robots. It allows us to run python scripts "in the cloud" on Google's servers, so you don't need to worry about installing software and dependencies on your own computer. 

For more details on using Colab, check out the [official introduction](https://colab.research.google.com/notebooks/basic_features_overview.ipynb). 

# Running Some Code

Colab notebooks consiste of text blocks (like this one) and code blocks (like the one below). Running code blocks executes the associated code, and prints the output below the block. 

In [ ]:
print("hello world")

You can modify code blocks as you wish to make interesting things happen. For example, try to complete the following function:

In [ ]:
def greeting():
    # Write some code here so that this function prints a friendly greeting

greeting()

# Saving Your code

Now that you've finished writing the `greeting` function above, you might like to save your work. There are a few ways to do this:

1. Save locally as an ipython notebook. Go to [File] --> [Download] --> [Dowload .ipynb]. Then you have a copy of this notebook saved to your local machine. You can then open it locally using software like [Jupyter](https://jupyter.org/), though some colab-specific features might not work. You can open it on colab by finding an existing colab notebook and going to [File] --> [Open Notebook] --> [Upload]

2. Save to Google Drive. This is as simple as going to [File] --> [Save a copy in Drive]. Once you have your own copy in Google Drive, you can save subsequent changes with Ctrl-S. 

Give that a try, and make sure your updated `greeting` function has been saved. 

# A Puzzle

Now that you're comfortable with the Colab interface, let's warm up our python skills with a classic puzzle:

Write a program that prints all of the integers between 0 and 100. If the number is divisible by 3, print "foo" instead of the number. If the number is divisible by 5, print "bar" instead of the number. If the number is divisible by 3 and 5, print "foobar". 

In [ ]:
# Write your foo-bar program here!